In [8]:
!pip install latex2sympy2 pandas 

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/

DEPRECATION: pytorch-lightning 1.7.6 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
spectral_aliases = {
    "R": (630, 690),
    "G": (520, 600),
    "B": (450, 520),
    # "G": 
    # "L":
    # "Y"
    "RED": (630, 690),
    "GREEN": (520, 600),
    "BLUE": (450, 520),
    "NIR": (760, 900),
    "rededge": (705, 745),
    # "MIR"
    "SWIR": (1550, 1750),
}

In [70]:
import re

class SpectrumNormalizer:
    def __init__(self, known_ranges):
        """
        known_ranges: dict[str, tuple[int, int]]
            Например: { "RED": (630, 690), "NIR": (760, 900) }
        """
        self.known = known_ranges

        # Компилируем регулярки
        self.re_nm = re.compile(r'(\d+)\s*nm')                 # 700nm
        self.re_range = re.compile(r'\[(\d+)\s*:\s*(\d+)\]')   # [1345:1400]
        self.re_name = re.compile(r'\b([A-Za-z_]+)\b')         # RED, NIR, etc.

    def normalize_single_nm(self, m):
        """ 700nm → [700:701] """
        v = int(m.group(1))
        return f'[{v}:{v+1}]'

    def normalize_range(self, m):
        """ [1345:1400] — проверяем и нормализуем """
        start = int(m.group(1))
        end = int(m.group(2))
        return f'[{start}:{end}]'

    def normalize_known_name(self, m):
        """ RED → [630:690] """
        name = m.group(1)
        if name in self.known:
            lo, hi = self.known[name]
            return f'[{lo}:{hi}]'
        return name  # оставляем нетронутым

    def remove_non_numeric_dots(self, s: str) -> str:
        """
        Удаляет точки, которые НЕ являются десятичными точками между цифрами.
        """
        # 1. Сохраняем те точки, которые являются десятичными
        # заменяем временно на спец-токен
        s = re.sub(r'(?<=\d)\.(?=\d)', '<DOT>', s)

        # 2. Удаляем все оставшиеся точки
        s = s.replace('.', '')

        # 3. Возвращаем десятичные точки обратно
        s = s.replace('<DOT>', '.')

        return s
    
    def replace_ranges_with_bands(self, latex_str, satellite_bands):
        """
        Заменяет все интервалы [a:b] на соответствующий бэнд спутника,
        который пересекается с этим диапазоном.
            
        :param latex_str: строка LaTeX с диапазонами вида [650:700]
        :param satellite_bands: словарь бэндов вида {"B1": [start, end], ...}
        :return: строка LaTeX с заменёнными диапазонами на бэнды
        """

        def find_band(a, b):
            for band, (start, end) in satellite_bands.items():
                # проверяем пересечение диапазонов
                if max(a, start) <= min(b, end):
                    return band
            return f"[{a}:{b}]"  # если пересечения нет, оставляем как есть

        # функция для замены
        def replacer(match):
            a, b = int(match.group(1)), int(match.group(2))
            return find_band(a, b)

        # регулярка для поиска [a:b]
        pattern = r'\[(\d+):(\d+)\]'
        return re.sub(pattern, replacer, latex_str)


    def normalize(self, latex: str, sat_bands: dict) -> str:
        """
        Полная нормализация LaTeX формулы со спектрами.
        Порядок:
        1. известные диапазоны
        2. диапазоны [...]
        3. единичные значения nm
        """

        s = latex

        # 1 — известные имена типа RED, NIR
        s = self.re_name.sub(self.normalize_known_name, s)

        # 2 — диапазоны [1345:1400]
        s = self.re_range.sub(self.normalize_range, s)

        # 3 — единичные длины 700nm
        s = self.re_nm.sub(self.normalize_single_nm, s)
        
        # 4 — удаляем лишние точки
        # s = self.remove_non_numeric_dots(s)

        # 4 - Подсавляем бэнды спутника
        s = self.replace_ranges_with_bands(s, sat_bands)
        return s


In [62]:
import os
import json

from pathlib import Path

root_dir = Path(str(os.getcwd)).resolve().parent.parent.parent
data_dir = root_dir / "data" / "sat_data" / "satellite_spectrums.json"


with open(data_dir, "r", encoding="utf-8") as f:
    bands = json.load(f)

In [73]:
import pandas as pd

df = pd.read_excel("index_db.xlsx")

In [74]:
normalier = SpectrumNormalizer(known_ranges=spectral_aliases)

for sat in bands:
    df[sat] = df['formula'].apply(lambda f: normalier.normalize(f, sat_bands=bands[sat]))
   

In [76]:
df.to_excel('new_index_db.xlsx', index=False, engine="openpyxl")

In [13]:
import pandas as pd

df = pd.read_excel("index_db.xlsx")

df["normalized_formula"] = df['formula'].apply(lambda f: normalizer.normalize(f))

In [11]:
df[["name", "abbreviation",	"formula", "normalized_formula", "variables",	"explanation_of_variables", "description", "wavelengths", "source",	"article_name",	"doi"]]

,name,abbreviation,formula,normalized_formula,variables,explanation_of_variables,description,wavelengths,source,article_name,doi
0,Atmospherically Resistant Vegetation Index,ARVI,\frac{NIR - RED - y ⁢ \left(\right. RED - BLUE...,\frac{[760:900] - [630:690] - y ⁢ \left(\right...,NIR = [781:1399],y = quotient derived from the components of at...,Minimizes the effects of atmospheric scatterin...,"Wavelengths420:480,640:760,781:1399",1.http://dx.doi.org/10.1080/02757259509532298\...,"1.Bannari, A.; Morin, D.; Bonn, F.; Huete, A. ...",1.10.1080/02757259509532298\n2.\n3.10.1016/s00...
1,Cellulose Absorption Index,CAI,100 ⁢ \left(\right. 0.5 ⁢ \left(\right. 2030nm...,100 ⁢ \left(\right. 0.5 ⁢ \left(\right. [2030:...,NaN,NaN,NaN,"Wavelengths2030,2100,2210",NaN,NaN,NaN
2,Normalized Difference NIR/Red Normalized Diffe...,NDVI,\frac{NIR - RED}{NIR + RED},\frac{[760:900] - [630:690]}{[760:900] + [630:...,"RED=[670;50;30],NIR=[800;10;10]",mit RED = 620 bis 700 nm (um 670nm) und NIR = ...,NaN,"Wavelengths670;50;30,800;10;10",1.http://dx.doi.org/10.1080/014311603100011696...,"1.Aparicio, N.; Villegas, D.; Royo, C.; Casade...",1.10.1080/0143116031000116967\n2.10.1080/02757...
3,Leaf Chlorophyll Index,LCI,\frac{[850] - [710]}{[850] + [680]},\frac{[850] - [710]}{[850] + [680]},NaN,NaN,NaN,"Wavelengths680,710,850",1.\n2.http://www.mdpi.com/1424-8220/8/6/3744/,"1.Datt, Bisun - TitleRemote Sensing of Water C...",1.http://dx.doi.org/10.1071/BT98042\n2.
4,Chlorophyll Absorption Ratio Index,CARI,\left(\right. \frac{700nm}{670nm} \left.\right...,\left(\right. \frac{[700:701]}{[670:671]} \lef...,"b=(550nm-((700nm-550nm)/150*550)), a=(700nm-55...",NaN,NaN,"Wavelengths550,670,700",1.\n2.\n3.http://www.sciencedirect.com/science...,"1.Broge, N. H.; Leblanc, E. - TitleComparing p...",1.10.1016/s0034-4257(00)00197-8\n2.\n3.10.1016...
...,...,...,...,...,...,...,...,...,...,...,...
318,Single Band 970,SB970,970nm,[970:971],NaN,NaN,NaN,Wavelengths970,1.http://www.sciencedirect.com/science/article...,"1.Curran, Paul J. - TitleRemote sensing of fol...",1.10.1016/0034-4257(89)90069-2
319,Three-Band Ratio 1200,RATIO1200,2 ⁢ \frac{[1180:1220]}{[1090:1110] + \left(\ri...,2 ⁢ \frac{[1180:1220]}{[1090:1110] + \left(\ri...,NaN,NaN,Estimate relative water content <60% at leaf l...,"Wavelengths1090:1110,1180:1220,1265:1285",1.http://dx.doi.org/10.1080/01431160210155965,"1.Pu, R.; Ge, S.; Kelly, N. M.; Gong, P. - Tit...",1.10.1080/01431160210155965
320,Simple Ratio 1058/1148,RVIhyp,\frac{1058nm}{1148nm},\frac{[1058:1059]}{[1148:1149]},NaN,NaN,Quantify LAI and water content at canopy level,"Wavelengths1058,1148",1.,"1.Schlerf, M.; Atzberger, C.; Hill, J. - Title...",1.
321,Normalized Difference 860/1640,SIWSI,\frac{860nm - 1640nm}{860nm + 1640nm},\frac{[860:861] - [1640:1641]}{[860:861] + [16...,NaN,NaN,"Estimate leaf of canopy water stress, especial...","Wavelengths860,1640",1.http://www.sciencedirect.com/science/article...,"1.Fensholt, Rasmus; Sandholt, Inge - TitleDeri...",1.10.1016/j.rse.2003.07.002


In [15]:
from latex2sympy2 import latex2sympy

expr = latex2sympy(r"\frac{700 - 670}{0.2*(700 - 550)}")
print(expr)
print(expr.simplify())

df["sympy_formula"] = df['formula'].apply(lambda f: normalizer.normalize(f))

(-670 + 700)/(((-550 + 700)/5))
1


In [33]:
import re
from sympy import symbols, sympify, simplify, sqrt

import re

def replace_ranges_with_bands(latex_str, satellite_bands):
    """
    Заменяет все интервалы [a:b] на соответствующий бэнд спутника,
    который пересекается с этим диапазоном.
    
    :param latex_str: строка LaTeX с диапазонами вида [650:700]
    :param satellite_bands: словарь бэндов вида {"B1": [start, end], ...}
    :return: строка LaTeX с заменёнными диапазонами на бэнды
    """

    def find_band(a, b):
        for band, (start, end) in satellite_bands.items():
            # проверяем пересечение диапазонов
            if max(a, start) <= min(b, end):
                return band
        return f"[{a}:{b}]"  # если пересечения нет, оставляем как есть

    # функция для замены
    def replacer(match):
        a, b = int(match.group(1)), int(match.group(2))
        return find_band(a, b)

    # регулярка для поиска [a:b]
    pattern = r'\[(\d+):(\d+)\]'
    return re.sub(pattern, replacer, latex_str)





latex = r"\frac{\left(\right. \left(\right. [700:701] - [670:671] \left.\right) - 0.2 ⁢ \left(\right. [700:701] - [550:551] \left.\right) \left.\right) ⁢ \left(\right. \frac{[700:701]}{[670:671]} \left.\right)}{\left(\right. 1.5 ⁢ \frac{1.2 ⁢ \left(\right. [800:801] - [550:551] \left.\right) - 2.5 ⁢ \left(\right. [670:671] - [550:551] \left.\right)}{\sqrt{\left(\left(\right. 2 ⁢ [800:801] + 1 \left.\right)\right)^{2} - \left(\right. 6 ⁢ [800:801] - 5 ⁢ \sqrt{[670:671]} \left.\right) - 0.5}} \left.\right)}"
expr = replace_ranges_with_bands(latex, satellite_bands={
        "B1": [450, 520],
        "B2": [530, 610],
        "B3": [630, 690],
        "B4": [780, 900],
        "B5": [1550, 1750],
        "B6": [2090, 2350],
        "B7": [1040, 1250],
        "B8": [520, 900]
    },)
print(expr)


\frac{\left(\right. \left(\right. B8 - B3 \left.\right) - 0.2 ⁢ \left(\right. B8 - B2 \left.\right) \left.\right) ⁢ \left(\right. \frac{B8}{B3} \left.\right)}{\left(\right. 1.5 ⁢ \frac{1.2 ⁢ \left(\right. B4 - B2 \left.\right) - 2.5 ⁢ \left(\right. B3 - B2 \left.\right)}{\sqrt{\left(\left(\right. 2 ⁢ B4 + 1 \left.\right)\right)^{2} - \left(\right. 6 ⁢ B4 - 5 ⁢ \sqrt{B3} \left.\right) - 0.5}} \left.\right)}


In [23]:
latex = r"\frac{[760:900] - [630:690] - y ⁢ \left(\right [630:690] - [450:520] \left\right)}{[760:900] + [630:690] - y ⁢ \left(\right [630:690] - [450:520] \left\right)}"

expr = latex_to_sympy_safe(latex)
print(expr)


SympifyError: Sympify of expression 'could not parse '\\frac{B760_900 - B630_690 - y * (right B630_690 - B450_520 (right)}{B760_900 + B630_690 - y * (right B630_690 - B450_520 (right)}'' failed, because of exception being raised:
TokenError: ('EOF in multi-line statement', (2, 0))